In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!head -n 10 '/content/drive/My Drive/StateData/wi.csv'

head: cannot open '/content/drive/My Drive/StateData/wi.csv' for reading: No such file or directory


In [ ]:
import pandas as pd
import numpy as np

# Initialize variables for aggregation
traffic_stops_per_date = {}

# Define chunk size for processing
chunk_size = 100000
file_path = "/content/drive/My Drive/StateData/wy.csv"  # Adjust the file path if necessary

# Process the file in chunks
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    # Convert the 'date' column to datetime
    chunk['date'] = pd.to_datetime(chunk['date'], errors='coerce')

    # Drop rows with invalid dates
    chunk = chunk.dropna(subset=['date'])

    # Group by date and count traffic stops
    chunk_counts = chunk.groupby('date').size()

    # Aggregate the results
    for date, count in chunk_counts.items():
        if date in traffic_stops_per_date:
            traffic_stops_per_date[date] += count
        else:
            traffic_stops_per_date[date] = count

# Convert the aggregated dictionary to a DataFrame
result = pd.DataFrame(list(traffic_stops_per_date.items()), columns=['date', 'traffic_stop_count'])

# Add the state column
result['state'] = 'WY'  # Vermont

# Save the processed data
output_path = "/content/drive/My Drive/StateData/processed_wy.csv"  # Adjust the output file path if necessary
result.to_csv(output_path, index=False)

# Display the processed data
print(result.head())


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/StateData/wy.csv'

In [ ]:
print(result.tail())

NameError: name 'result' is not defined

In [ ]:
import pandas as pd
import os

# Step 1: Combine all CSVs
directory = "/content/drive/My Drive/StateData/"  # Directory where all processed CSVs are stored
all_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.startswith("processed_")]

# Read and combine all files
df_list = [pd.read_csv(file) for file in all_files]
data = pd.concat(df_list, ignore_index=True)

# Ensure 'date' column is in datetime format
data['date'] = pd.to_datetime(data['date'])

# Step 2: Group by state and date, calculate cumulative traffic stops
data = data.sort_values(by=['state', 'date'])  # Sort by state and date
data['cumulative_traffic_stops'] = data.groupby('state')['traffic_stop_count'].cumsum()

# Save as CSV
output_csv_path = "/content/drive/My Drive/StateData/combined_traffic_data.csv"
data.to_csv(output_csv_path, index=False)

# Verify saved file
print(f"Data saved to {output_csv_path}")


Data saved to /content/drive/My Drive/StateData/combined_traffic_data.csv


In [ ]:
# Group by state and count the number of rows per state
rows_per_state = data.groupby('state').size()

# Display the result
print(rows_per_state)


# Convert to a DataFrame and sort by the count
rows_per_state_df = rows_per_state.reset_index(name='row_count').sort_values(by='row_count', ascending=False)

# Display the result
print(rows_per_state_df)


state
CA    2545
CO    2921
CT     731
GA    1827
IA    3768
IL    2192
MA    3287
MD     459
MO       6
MS    1297
MT    3207
NC    5844
ND    2002
NE      60
NH     730
NJ    2922
NV    1324
NY    2905
OH    2559
OR       5
RI    3809
SC    4228
SD    1308
TN    5769
VA     538
VT    2010
WI    2072
WY     731
dtype: int64
   state  row_count
11    NC       5844
23    TN       5769
21    SC       4228
20    RI       3809
4     IA       3768
6     MA       3287
10    MT       3207
15    NJ       2922
1     CO       2921
17    NY       2905
18    OH       2559
0     CA       2545
5     IL       2192
26    WI       2072
25    VT       2010
12    ND       2002
3     GA       1827
16    NV       1324
22    SD       1308
9     MS       1297
27    WY        731
2     CT        731
14    NH        730
24    VA        538
7     MD        459
13    NE         60
8     MO          6
19    OR          5


In [3]:
import pandas as pd

# Load the dataset
data_path = "/content/drive/My Drive/StateData/combined_traffic_data.parquet"
data = pd.read_parquet(data_path)

# Ensure 'date' is in datetime format
data['date'] = pd.to_datetime(data['date'])

# Filter out dates before February 20, 2004
start_date = pd.Timestamp("2004-02-20")
filtered_data = data[data['date'] >= start_date]

# Add a 'week' column (start of the week)
filtered_data['week'] = filtered_data['date'].dt.to_period('W').apply(lambda r: r.start_time)

# Group by state and week, and sum traffic stops for each week
weekly_data = filtered_data.groupby(['state', 'week'], as_index=False).agg({
    'traffic_stop_count': 'sum',
    'cumulative_traffic_stops': 'max'  # Use the max cumulative value for each week
})

# Save the weekly aggregated data to both CSV and Parquet files
output_csv_path = "/content/drive/My Drive/StateData/weekly_traffic_data.csv"
output_parquet_path = "/content/drive/My Drive/StateData/weekly_traffic_data.parquet"

# Save as CSV
weekly_data.to_csv(output_csv_path, index=False)

# Save as Parquet
weekly_data.to_parquet(output_parquet_path, index=False)

print(f"Weekly aggregated data saved to CSV at {output_csv_path}")
print(f"Weekly aggregated data saved to Parquet at {output_parquet_path}")


<ipython-input-3-30deaa9bd9f3>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['week'] = filtered_data['date'].dt.to_period('W').apply(lambda r: r.start_time)


Weekly aggregated data saved to CSV at /content/drive/My Drive/StateData/weekly_traffic_data.csv
Weekly aggregated data saved to Parquet at /content/drive/My Drive/StateData/weekly_traffic_data.parquet


In [4]:
pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3


In [5]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# Load the weekly aggregated data from the .parquet file
data_path = "/content/drive/My Drive/StateData/weekly_traffic_data.parquet"
data = pd.read_parquet(data_path)

# Ensure 'week' column is in datetime format
data['week'] = pd.to_datetime(data['week'])

# Calculate the total number of weeks
total_weeks = len(data['week'].unique())

# Initialize the Dash app
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.H1("Interactive Weekly Cumulative Traffic Stops Map"),
    dcc.Graph(id="choropleth-map"),
    dcc.Slider(
        id="week-slider",
        min=0,
        max=total_weeks - 1,
        step=1,
        value=total_weeks - 1,  # Start from the most recent week
        tooltip={"placement": "bottom", "always_visible": False},  # Disable default tooltip
        marks={},  # Remove all marks
        updatemode='drag'
    ),
    html.Div(id="slider-label", style={"textAlign": "center", "marginTop": "20px", "fontSize": "18px"})
])

# Callback to update the slider label dynamically
@app.callback(
    Output("slider-label", "children"),
    [Input("week-slider", "value")]
)
def update_slider_label(selected_week_index):
    # Calculate "years and weeks ago"
    weeks_ago = total_weeks - selected_week_index - 1
    years = weeks_ago // 52
    weeks = weeks_ago % 52

    if weeks_ago == 0:
        return "This week"
    elif years > 0:
        return f"{years} year{'s' if years > 1 else ''} and {weeks} week{'s' if weeks > 1 else ''} ago"
    else:
        return f"{weeks} week{'s' if weeks > 1 else ''} ago"

# Callback to update the map based on the selected week
@app.callback(
    Output("choropleth-map", "figure"),
    [Input("week-slider", "value")]
)
def update_map(selected_week_index):
    # Filter the data up to the selected week
    selected_week = sorted(data['week'].unique())[selected_week_index]
    filtered_data = data[data['week'] <= selected_week]

    # Take the latest cumulative traffic stops for each state up to the selected week
    filtered_data = filtered_data.groupby('state').last().reset_index()

    # Create the choropleth map with a more drastic color scheme
    fig = px.choropleth(
        filtered_data,
        locations="state",
        locationmode="USA-states",
        color="cumulative_traffic_stops",
        hover_name="state",
        scope="usa",
        title=f"Cumulative Traffic Stops up to {selected_week.strftime('%Y-%m-%d')}",
        color_continuous_scale="Plasma"  # Use a more vibrant color scheme
    )
    return fig

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

In [6]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output, callback_context
from datetime import datetime
import dash

# Initialize the Dash app
app = Dash(__name__)

# Load the weekly aggregated data from the .parquet file
data_path = "/content/drive/My Drive/StateData/weekly_traffic_data.parquet"
data = pd.read_parquet(data_path)

# Ensure 'week' column is in datetime format and sorted
data['week'] = pd.to_datetime(data['week'])
data = data.sort_values('week').reset_index(drop=True)

# Get sorted unique weeks as a list
unique_weeks = sorted(data['week'].unique())
total_weeks = len(unique_weeks)

# Function to find the index of the start date
def find_start_index(start_date):
    if start_date in unique_weeks:
        return unique_weeks.index(start_date)
    else:
        # Find the next available week after the start_date
        for i, week in enumerate(unique_weeks):
            if week > start_date:
                return i
        # If no week is found after start_date, return the last index
        return total_weeks - 1

# App layout
app.layout = html.Div([
    html.H1("Interactive Weekly Cumulative Traffic Stops Map"),

    # Date Picker for Start Date
    html.Div([
        html.Label("Select Start Date:"),
        dcc.DatePickerSingle(
            id='start-date-picker',
            min_date_allowed=min(unique_weeks),  # Use min() instead of unique_weeks.min()
            max_date_allowed=max(unique_weeks),  # Use max() instead of unique_weeks.max()
            initial_visible_month=min(unique_weeks),  # Use min() instead of unique_weeks.min()
            date=min(unique_weeks).date()  # Use min() instead of unique_weeks.min()
        )
    ], style={"marginBottom": "20px"}),

    dcc.Graph(id="choropleth-map"),

    # Slider for Weeks since Start Date
    dcc.Slider(
        id="week-slider",
        min=0,
        max=total_weeks - 1,
        step=1,
        value=total_weeks - 1,  # Start from the most recent week
        tooltip={"placement": "bottom", "always_visible": False},
        marks={},  # Marks will be dynamically generated
        updatemode='drag'
    ),

    html.Div(id="slider-label", style={"textAlign": "center", "marginTop": "20px", "fontSize": "18px"})
])

# Callback to update the slider based on the selected start date
@app.callback(
    [Output("week-slider", "min"),
     Output("week-slider", "max"),
     Output("week-slider", "marks"),
     Output("week-slider", "value")],
    [Input("start-date-picker", "date")]
)
def update_slider(start_date):
    if start_date is None:
        # If no date is selected, set to default min date
        default_start_date = min(unique_weeks)
    else:
        try:
            # Convert start_date to datetime
            start_date = pd.to_datetime(start_date)
            default_start_date = start_date
        except Exception:
            # If invalid date, set to default
            default_start_date = min(unique_weeks)

    try:
        # Find the index of the start_date in unique_weeks
        start_index = find_start_index(default_start_date)

        # Calculate the number of weeks from start_date to the present
        max_index = total_weeks - 1
        num_weeks = max_index - start_index + 1

        if num_weeks <= 0:
            # If start_index is beyond max, set to min and max as zero
            return 0, 0, {0: unique_weeks[max_index].strftime('%Y-%m-%d')}, 0

        # Generate marks every 10 weeks for readability
        marks = {}
        interval = max(1, num_weeks // 10)
        for i in range(start_index, max_index + 1, interval):
            week_label = unique_weeks[i].strftime('%Y-%m-%d')
            marks[i - start_index] = week_label

        # Ensure the last week is always marked
        if (max_index - start_index) % interval != 0:
            marks[num_weeks - 1] = unique_weeks[max_index].strftime('%Y-%m-%d')

        # Set the initial slider value to the latest week in the selected range
        slider_value = num_weeks - 1

        return 0, num_weeks - 1, marks, slider_value
    except Exception as e:
        # Log the exception and prevent the callback from updating outputs
        print(f"Error in update_slider: {e}")
        return dash.no_update, dash.no_update, dash.no_update, dash.no_update

# Callback to update the slider label based on the selected week
@app.callback(
    Output("slider-label", "children"),
    [Input("week-slider", "value"),
     Input("start-date-picker", "date")]
)
def update_slider_label(selected_week_offset, start_date):
    if start_date is None:
        # If no date is selected, use the default min date
        default_start_date = min(unique_weeks)
    else:
        try:
            # Convert start_date to datetime
            default_start_date = pd.to_datetime(start_date)
        except Exception:
            # If invalid date, use the default min date
            default_start_date = min(unique_weeks)

    try:
        # Find the index of the start_date in unique_weeks
        start_index = find_start_index(default_start_date)

        # Calculate the selected week index
        selected_week_index = start_index + selected_week_offset
        selected_week_index = min(selected_week_index, total_weeks - 1)  # Ensure it doesn't exceed

        selected_week = unique_weeks[selected_week_index]

        # Calculate "years and weeks ago" from the selected week to the present
        weeks_ago = total_weeks - (selected_week_index + 1)
        years = weeks_ago // 52
        weeks = weeks_ago % 52

        if weeks_ago == 0:
            return "This week"
        elif years > 0:
            return f"{years} year{'s' if years > 1 else ''} and {weeks} week{'s' if weeks > 1 else ''} ago"
        else:
            return f"{weeks} week{'s' if weeks > 1 else ''} ago"
    except Exception as e:
        # Log the exception and prevent the callback from updating the output
        print(f"Error in update_slider_label: {e}")
        return "Error updating label"

@app.callback(
    Output("choropleth-map", "figure"),
    [Input("week-slider", "value"),
     Input("start-date-picker", "date")]
)
def update_map(selected_week_offset, start_date):
    if start_date is None:
        # If no date is selected, use the default min date
        default_start_date = min(unique_weeks)
    else:
        try:
            # Convert start_date to datetime
            default_start_date = pd.to_datetime(start_date)
        except Exception:
            # If invalid date, use the default min date
            default_start_date = min(unique_weeks)

    try:
        # Find the index of the start_date in unique_weeks
        start_index = find_start_index(default_start_date)

        # Calculate the selected week index
        selected_week_index = start_index + selected_week_offset
        selected_week_index = min(selected_week_index, total_weeks - 1)  # Ensure it doesn't exceed

        selected_week = unique_weeks[selected_week_index]

        # Filter the data up to the selected week
        filtered_data = data[data['week'] <= selected_week]

        # Take the latest cumulative traffic stops for each state up to the selected week
        filtered_data = filtered_data.groupby('state').last().reset_index()

        # Create a custom hover text column
        filtered_data['hover_text'] = (
            "State: " + filtered_data['state'] +
            "<br>Cumulative Traffic Stops: " + filtered_data['cumulative_traffic_stops'].apply(lambda x: f"{x:,.0f}")
        )

        # Create the choropleth map with custom hover text
        fig = px.choropleth(
            filtered_data,
            locations="state",
            locationmode="USA-states",
            color="cumulative_traffic_stops",
            hover_name="state",
            scope="usa",
            title=f"Cumulative Traffic Stops up to {selected_week.strftime('%Y-%m-%d')}",
            color_continuous_scale="Plasma",
            hover_data={'hover_text': True},  # Use only custom hover text
        )

        # Update the layout to only show the custom hover text
        fig.update_traces(hovertemplate='%{customdata[0]}<extra></extra>')
        fig.update_layout(transition_duration=500)
        return fig
    except Exception as e:
        # Log the exception and return an empty figure with an error message
        print(f"Error in update_map: {e}")
        return {
            "data": [],
            "layout": {
                "title": "Error updating the map.",
                "xaxis": {"visible": False},
                "yaxis": {"visible": False},
            }
        }

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)


<IPython.core.display.Javascript object>